# Scratch

---

### Imports & Setup

Installs

In [8]:
# Run this cell to install required packages in your Jupyter notebook environment
!pip install sqlalchemy sqlmodel psycopg2-binary pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 66.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 45.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 42.6 MB/s eta 0:00:00


Connect to DB

In [11]:
from sqlalchemy import create_engine
from sqlmodel import Session, SQLModel, select

# PostgreSQL connection URL
DATABASE_HOST = "db"  # Service name in docker-compose.yml
DATABASE_PORT = "5432"
DATABASE_NAME = "updatesdb"
DATABASE_USER = "postgres"
DATABASE_PASSWORD = "your_password"

ENGINE_URL = f"postgresql://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"

engine = create_engine(ENGINE_URL, echo=True, pool_pre_ping=True)

# Optional: If you need to create the tables in your database (uncomment if necessary)
# SQLModel.metadata.create_all(engine)

Models

In [6]:
from typing import Optional
from sqlmodel import SQLModel, Field, Relationship
from sqlalchemy import Column, BigInteger

class ChatType(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    type_name: str = Field(sa_column_kwargs={"unique": False})

class Chat(SQLModel, table=True):
    id: int = Field(primary_key=True)
    chat_id: Optional[int] = Field(default=None, sa_column=Column(BigInteger()))
    all_members_are_administrators: bool = Field()
    title: str = Field()
    type_id: int = Field(foreign_key="chattype.id")
    type: ChatType = Relationship()

class User(SQLModel, table=True):
    id: int = Field(primary_key=True, index=True)
    user_id: Optional[int] = Field(default=None)  # Telegram user ID
    first_name: str = Field()
    last_name: Optional[str] = Field(default=None)
    is_bot: bool = Field()
    language_code: str = Field()
    username: Optional[str] = Field()

class Message(SQLModel, table=True):
    id: int = Field(primary_key=True)
    message_id: int = Field(default=None)
    channel_chat_created: bool = Field()
    chat_id: int = Field(foreign_key="chat.id")
    chat: Chat = Relationship()
    date: int = Field()
    delete_chat_photo: bool = Field()
    from_user_id: int = Field(foreign_key="user.id")
    from_user: User = Relationship()
    group_chat_created: bool = Field()
    reply_to_message_id: Optional[int] = Field(default=None, foreign_key="message.id")
    reply_to_message: 'Message' = Relationship(sa_relationship_kwargs={"remote_side": "Message.id"})
    supergroup_chat_created: bool = Field()
    text: Optional[str] = Field()

class Update(SQLModel, table=True):
    update_id: int = Field(primary_key=True)
    message_id: Optional[int] = Field(default=None, foreign_key="message.id")
    message: Optional[Message] = Relationship()

---

### Display Tables (in pandas)

In [12]:
import pandas as pd

def display_table_data_pandas(model_class):
    with Session(engine) as session:
        statement = select(model_class)
        results = session.exec(statement).all()
        return pd.DataFrame([result.__dict__ for result in results])

In [14]:
# Display data using pandas for better formatting
display_table_data_pandas(Message)

2024-05-06 10:48:46,712 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-06 10:48:46,715 INFO sqlalchemy.engine.Engine SELECT message.id, message.message_id, message.channel_chat_created, message.chat_id, message.date, message.delete_chat_photo, message.from_user_id, message.group_chat_created, message.reply_to_message_id, message.supergroup_chat_created, message.text 
FROM message
2024-05-06 10:48:46,715 INFO sqlalchemy.engine.Engine [generated in 0.00046s] {}
2024-05-06 10:48:46,718 INFO sqlalchemy.engine.Engine ROLLBACK


,_sa_instance_state,message_id,chat_id,date,from_user_id,group_chat_created,id,channel_chat_created,delete_chat_photo,reply_to_message_id,supergroup_chat_created,text
0,<sqlalchemy.orm.state.InstanceState object at ...,180,1,1714930267,1,False,1,False,False,NaN,False,"Earth in forgetful snow, feeding"
1,<sqlalchemy.orm.state.InstanceState object at ...,180,3,1714930267,3,False,2,False,False,NaN,False,"Earth in forgetful snow, feeding"
2,<sqlalchemy.orm.state.InstanceState object at ...,181,2,1714930290,2,False,3,False,False,2.0,False,A little life with dried tubers.
3,<sqlalchemy.orm.state.InstanceState object at ...,180,5,1714930267,5,False,4,False,False,NaN,False,"Earth in forgetful snow, feeding"
4,<sqlalchemy.orm.state.InstanceState object at ...,182,4,1714930308,4,False,5,False,False,4.0,False,"Summer surprised us, coming over the Starnberg..."
5,<sqlalchemy.orm.state.InstanceState object at ...,182,7,1714930308,7,False,6,False,False,NaN,False,"Summer surprised us, coming over the Starnberg..."
6,<sqlalchemy.orm.state.InstanceState object at ...,183,6,1714930323,6,False,7,False,False,6.0,False,With a shower of rain; we stopped in the colon...
7,<sqlalchemy.orm.state.InstanceState object at ...,184,8,1714930347,8,False,8,False,False,NaN,False,"And went on in sunlight, into the Hofgarten,"
8,<sqlalchemy.orm.state.InstanceState object at ...,185,9,1714990843,9,False,9,False,False,NaN,False,"And drank coffee, and talked for an hour."
